[View in Colaboratory](https://colab.research.google.com/github/ostegm/serve_keras_model/blob/master/Train_And_Serve_Keras_Model.ipynb)

# Train and Serve a Keras Model


In [0]:
import tensorflow as tf
import json
import numpy as np
import os
import pandas as pd
import requests

## Data
- For this example, we'll predict the median house value (in thousands) for various city blocks in california. 
- For simplicity, we'll use only two features, however better models can be made from these features.

In [12]:
df = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv", sep=",")
df = df.reindex(np.random.permutation(df.index))
df["median_house_value"] /= 1000.0
df["avg_rooms_per_person"] = df.total_bedrooms / df.population

split_idx = 13000
FEATURE_NAMES = ['avg_rooms_per_person', 'median_income']


train_ftrs = df.loc[:split_idx, FEATURE_NAMES]
train_labels = df.loc[:split_idx, 'median_house_value']
test_ftrs = df.loc[split_idx: , FEATURE_NAMES]
test_labels = df.loc[split_idx: , 'median_house_value']
print(train_ftrs.shape, test_ftrs.shape)

((13089, 2), (3912, 2))


## Build and Train  a Keras Model

In [0]:
input_ftrs = tf.keras.layers.Input(shape=(len(FEATURE_NAMES),))
dense = tf.keras.layers.Dense(128, activation='relu')(input_ftrs)
pred = tf.keras.layers.Dense(1, activation='linear')(dense)
model = tf.keras.models.Model(inputs=[input_ftrs], outputs=pred)
model.compile(loss='mean_squared_error', optimizer='adam')

In [22]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 2)                 0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               384       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 129       
Total params: 513
Trainable params: 513
Non-trainable params: 0
_________________________________________________________________


In [23]:
history = model.fit(train_ftrs, train_labels,
        validation_data=(test_ftrs, test_labels),
        epochs=10,
        batch_size=128)

Train on 13089 samples, validate on 3912 samples
Epoch 1/10
13089/13089 [==============================] - 0s 31us/step - loss: 55079.9102 - val_loss: 52525.3370
Epoch 2/10
13089/13089 [==============================] - 0s 19us/step - loss: 49190.0092 - val_loss: 43975.0623
Epoch 3/10
13089/13089 [==============================] - 0s 18us/step - loss: 38465.6675 - val_loss: 31538.7996
Epoch 4/10
13089/13089 [==============================] - 0s 15us/step - loss: 25795.9812 - val_loss: 19557.8015
Epoch 5/10
13089/13089 [==============================] - 0s 15us/step - loss: 15568.1110 - val_loss: 11621.7387
Epoch 6/10
13089/13089 [==============================] - 0s 14us/step - loss: 9811.1705 - val_loss: 8065.7439
Epoch 7/10
13089/13089 [==============================] - 0s 15us/step - loss: 7565.5264 - val_loss: 7024.6498
Epoch 8/10
13089/13089 [==============================] - 0s 19us/step - loss: 7001.6028 - val_loss: 6849.4734
Epoch 9/10
13089/13089 [=============================

## Build and Train the same Keras Model as a TF Estimator

- Only change required is naming our input(s) - which will help when serving

In [0]:
# Note we need to name our input layer for use with estimator serving function.
input_ftrs = tf.keras.layers.Input(shape=(len(FEATURE_NAMES),), name='input_features')
dense = tf.keras.layers.Dense(128, activation='relu')(input_ftrs)
pred = tf.keras.layers.Dense(1, activation='linear')(dense)
model = tf.keras.models.Model(inputs=[input_ftrs], outputs=pred)
model.compile(loss='mean_squared_error', optimizer='adam')

In [43]:
estimator = tf.keras.estimator.model_to_estimator(model, model_dir='./my_trained_model')

INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x118326d10>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': './my_trained_model', '_save_summary_steps': 100}


## Create an Estimator input function

To pipe data into Estimators we need to define a data importing function like shown below which returns batches of `(images, labels)` as shown below:

In [0]:
!rm -rf './my_trained_model/'

In [0]:
def input_fn(features, labels, epochs=1, batch_size=128, shuffle=False):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((features, labels))

    # Shuffle, repeat, and batch the examples.
    if shuffle:
      dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size)

    # Return the dataset.
    return dataset

In [0]:

# Training input on the whole training set with for 10 epochs
train_input_fn = lambda:input_fn(
    features=train_ftrs,
    labels=train_labels,
    epochs=5,
)
    

# Prediction on the test set - notice we don't repeat the dataset.
test_input_fn = lambda:input_fn(
    features=test_ftrs,
    labels=test_labels,
)

## Train and Evaluate

In [50]:
train_spec = tf.estimator.TrainSpec(train_input_fn, max_steps=20000)
eval_spec = tf.estimator.EvalSpec(test_input_fn)
tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 600 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into ./my_trained_model/model.ckpt.
INFO:tensorflow:loss = 57138.625, step = 1
INFO:tensorflow:global_step/sec: 255.984
INFO:tensorflow:loss = 45000.934, step = 101 (0.392 sec)
INFO:tensorflow:global_step/sec: 479.72
INFO:tensorflow:loss = 47359.582, step = 201 (0.209 sec)
INFO:tensorflow:global_step/sec: 513.463
INFO:tensorflow:loss = 34048.914, step = 301 (0.194 sec)
INFO:tensorflow:global_step/sec: 403.074
INFO:tensorflow:loss = 20972.074, step = 401 (0.248 sec)
INFO:tensorflow:global_step/sec: 460.55

## Export with a JSON serving function

In [0]:
!rm -rf /tmp/models
!mkdir -p /tmp/models

In [66]:
def serving_input_receiver_fn():
    receiver_tensors = {}
    for ftr_name in FEATURE_NAMES:
      receiver_tensors[ftr_name] = tf.placeholder(tf.float32, [None, 1])

    # Concat all input features for our Keras input layer.
    concat = [receiver_tensors[f] for f in FEATURE_NAMES]
    # Note- the key here must match our Keras input layer's name above
    features = {
      'input_features': tf.concat(concat, axis=1)
    }
    return tf.estimator.export.ServingInputReceiver(
        receiver_tensors=receiver_tensors,
        features=features)
  
estimator.export_savedmodel('/tmp/models/demo', serving_input_receiver_fn=serving_input_receiver_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tensorflow:Restoring parameters from ./my_trained_model/model.ckpt-20000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /tmp/models/demo/temp-1537393582/saved_model.pb


'/tmp/models/demo/1537393582'

In [67]:
!ls /tmp/models/demo

1537393377 1537393582


## Serve model as a rest API on your local machine
- Note: I'm using docker here, but you can get the tf serving binary several ways. 
- Also easy to serve a model from GCP CloudML.

In [0]:
# Pull docker image: https://hub.docker.com/r/tensorflow/serving/
!docker pull tensorflow/serving

In [70]:
!docker run -p 8501:8501 --name tfserving --mount type=bind,source=/tmp/models/demo,target=/models/demo -e MODEL_NAME=demo -t tensorflow/serving &> logs.txt

^C


In [71]:
# Construct single example matching our input signature:
single_example = {
    'avg_rooms_per_person': [0.0],
    'median_income': [10.0],
}


# Get predictions from REST api via requests module:
endpoint = 'http://localhost:8501/v1/models/demo:predict'
data = {
    'instances': [single_example]}
r = requests.post(endpoint, data=json.dumps(data))
predictions = json.loads(r.content)
predictions

{u'predictions': [[207.861]]}

In [72]:
# Similarly, you could batch multiple instances:
data = {
    'instances': [single_example, single_example, single_example]}

r = requests.post(endpoint, data=json.dumps(data))
predictions = json.loads(r.content)
predictions

{u'predictions': [[207.861], [207.861], [207.861]]}

## Kill docker containter

In [73]:
!docker kill tfserving && docker rm tfserving

tfserving
tfserving
